## 1. Set-up

In [1]:
# Mount Drive files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
MODEL_VERSION = "v1"

In [3]:
# https://blog.tensorflow.org/2019/11/hugging-face-state-of-art-natural.html
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 54.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 21.9 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd

import transformers
from datasets import Dataset,load_dataset,load_from_disk, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForSequenceClassification, BertTokenizer, Trainer
from scipy.stats import pearsonr
import sklearn.metrics
from datasets import load_metric
from sklearn.metrics import mean_squared_error

## 2. Arrange datasets and tokenize them

The main difference of this experiment is that train_df + dev_df are joined to produce one training dataset. And test_df becomes the validation/development dataset.

In [10]:
initial_train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dl4nlp_labs/final_project/sentence_similarity/data/en-test.txt", sep="\t")
initial_dev_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dl4nlp_labs/final_project/sentence_similarity/data/en-val.txt", sep="\t")
print(initial_train_df.shape)
#train_df = initial_train_df.append(initial_dev_df, ignore_index=True)
#train_df

(249, 3)


In [12]:
initial_dev_df

,an alarm in the offing on # climatechange # extremeweather # globalwarming # eco,an alarm in the offing on climate change,4.6
0,az gov jan brewer changes her mind and will ex...,national briefing | southwest : arizona : gove...,3.6
1,"amazing , you can play violent shooting games ...",nra draws heat for shooting game,3.6
2,a girl in a pink shirt is sprawling her limbs ...,a man dressed in a santa suit is climbing up a...,0.0
3,when the switch is on a closed path with the bulb,when the switch and the bulb are contained in ...,3.6
4,"if you have access to it , you can very easily...",you can have a look at the arxiv cultoromics w...,0.2
...,...,...,...
1494,dewhurst 's proposal calls for an abrupt end t...,the committee would propose a replacement for ...,2.6
1495,"start abruptly , ignite",start abruptly .,3.0
1496,the act of annoying someone or something,the act of projecting out from something .,0.2
1497,the debate is closed .,in the debate is closed .,4.6


In [ ]:
text_list_train = train_df['MTTargetSegment'] + ' [SEP] ' + train_df['SourceSegment'] + ' [SEP] ' + train_df['Post-editedTarget']
#text_list_dev = dev_df['MTTargetSegment'] + ' [SEP] ' + dev_df['SourceSegment'] + ' [SEP] ' + dev_df['Post-editedTarget'] 

X_train = pd.DataFrame(zip(text_list_train, train_df["TER7"]), columns = ["text", "label"])
#X_dev = pd.DataFrame(zip(text_list_dev, dev_df["TER7"]), columns = ["text", "label"])

In [ ]:
dev_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/MT-model/data/validation_only.csv", encoding = "windows-1252")

text_list_dev = dev_df['MTTargetSegment'] + ' [SEP] ' + dev_df['SourceSegment'] + ' [SEP] ' + dev_df['Post-editedTarget'] 

X_dev = pd.DataFrame(zip(text_list_dev, dev_df["TER7"]), columns = ["text", "label"])

X_dev

,text,label
0,“Orain ni Barbie bat eta zu nintzen Hot Wheels...,40.000
1,“Orain ni Barbie bat eta zu nintzen Hot Wheels...,17.647
2,“Orain ni Barbie bat eta zu nintzen Hot Wheels...,12.500
3,“Orain ni Barbie bat eta zu nintzen Hot Wheels...,35.000
4,“Orain ni Barbie bat eta zu nintzen Hot Wheels...,17.647
...,...,...
588,Uretako txorrota irekitzea. [SEP] Abrir el gri...,25.000
589,Atea irekitzea eta arropa erretiratzea. [SEP] ...,0.000
590,Garbiketarako gobadaren prestaketa [SEP] Prepa...,0.000
591,Etiketako seinaleak: [SEP] Indicaciones en la ...,80.000


In [ ]:
# Random test/dev partition

# dataset_p = Dataset.from_pandas(X_train,preserve_index=False) 
# dataset = dataset_p.train_test_split(test_size=0.3) 
# dataset

In [ ]:
# Custom test/dev partition

dataset_train = Dataset.from_pandas(X_train,preserve_index=False) 
dataset_dev = Dataset.from_pandas(X_dev,preserve_index=False) 
dataset = DatasetDict({"train": dataset_train,
                     "test": dataset_dev})
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4904
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 593
    })
})

In [ ]:
#model = AutoModelForSequenceClassification.from_pretrained("ixa-ehu/berteus-base-cased", num_labels=1)
#tokenizer = AutoTokenizer.from_pretrained("ixa-ehu/berteus-base-cased")

#model = BertForSequenceClassification.from_pretrained("ixa-ehu/ixambert-base-cased", num_labels=1)
#tokenizer = BertTokenizer.from_pretrained("ixa-ehu/ixambert-base-cased", model_max_length=512)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased", num_labels=1)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased", model_max_length=512)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier

Tokenize datasets

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/4904 [00:00<?, ? examples/s]

Map:   0%|          | 0/593 [00:00<?, ? examples/s]

We use rmse as a metric (the lower this value, the better the model performs)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

In [ ]:
MODEL_VERSION

'v8'

## 3. Train the model

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs = 20,
                                  save_total_limit = 2,
                                  save_strategy = 'no',
                                  #load_best_model_at_end=False
                                  load_best_model_at_end=False
                                  )



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4904
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6140
  Number of trainable parameters = 65782273


Epoch,Training Loss,Validation Loss,Rmse
1,1267.459200,619.625977,24.892288
2,416.184100,213.061340,14.596621
3,204.718900,189.378464,13.761484
4,142.362300,217.968796,14.763766
5,114.211000,151.340195,12.302040
6,90.819200,159.328598,12.622543
7,77.605900,160.028564,12.650240
8,66.535600,158.368591,12.584458
9,59.326200,136.148087,11.668251
10,51.401300,139.335815,11.804059


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 593
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 593
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 59

TrainOutput(global_step=6140, training_loss=140.54750014314433, metrics={'train_runtime': 4828.7262, 'train_samples_per_second': 20.312, 'train_steps_per_second': 1.272, 'total_flos': 1.299217075912704e+16, 'train_loss': 140.54750014314433, 'epoch': 20.0})

## 4. Save model

In [ ]:
# save the model/tokenizer
model.save_pretrained(f"/content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/model_{MODEL_VERSION}")
tokenizer.save_pretrained(f"/content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/tokenizer_{MODEL_VERSION}")


Configuration saved in /content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/model_v8/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/model_v8/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/tokenizer_v8/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/tokenizer_v8/special_tokens_map.json


('/content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/tokenizer_v8/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/tokenizer_v8/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/tokenizer_v8/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/tokenizer_v8/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/tokenizer_v8/tokenizer.json')

In [ ]:

# load the model/tokenizer
model = AutoModelForSequenceClassification.from_pretrained(f"/content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/model_{MODEL_VERSION}")
tokenizer = AutoTokenizer.from_pretrained(f"/content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/tokenizer_{MODEL_VERSION}")



loading configuration file /content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/model_v8/config.json
Model config DistilBertConfig {
  "_name_or_path": "/content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/model_v8",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "problem_type": "regression",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "vocab_size": 28996
}

loading weights file /content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/model_v8/pytorch_m

## 5. New predictions

In [ ]:
validation_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/MT-model/data/test2020.csv", encoding = "windows-1252")
validation_df

text_list_validation = validation_df['MTTargetSegment'] + ' [SEP] ' + validation_df['SourceSegment'] 

X_validation = pd.DataFrame(zip(text_list_validation, validation_df["TER7"]), columns = ["text", "label"])

X_validation

,text,label
0,"Hots, estereotipoek aurrez asmatutako ideiaz h...",62.069
1,Rol horiek haurtzarotik barneratzen dira; bain...,5.556
2,Eta hauste horiek zigortuko dira (“hori nesken...,85.294
3,Printzeak printzesa salbatzen du. Nahiz eta ip...,0.000
4,“Edurnezuriren printzea hartaz maitemintzen da...,0.000
...,...,...
707,"Markagailuak, historia eta gordetako orrialdea...",11.111
708,Hasiera-orria ezartzea azken funtzio hau baime...,35.484
709,"Googlen bilatzeko, Googleko Bilaketa-Barran sa...",12.500
710,Saka dezan egiaztatzeko joatean. [SEP] Pulse e...,60.000


In [ ]:
trainer = Trainer(model=model)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

def pipeline_prediction(texts):
    df=pd.DataFrame({'text':texts})
    dataset = Dataset.from_pandas(df,preserve_index=False) 
    tokenized_datasets = dataset.map(tokenize_function)
    raw_pred, _, _ = trainer.predict(tokenized_datasets) 
    return(raw_pred)

def evaluate(preds, gold):
    # Compute pearson:
    pearson = pearsonr(preds, gold)[0]
      
    # Compute rmse
    rmse = mean_squared_error(preds, gold, squared=False)

    print(f"RMSE = {rmse:.6f}")
    print(f"Pearson: {pearson:.6f}")

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
final_preds = pipeline_prediction(list(X_validation["text"]))
pred_list = [pred[0] for pred in final_preds]

Map:   0%|          | 0/712 [00:00<?, ? examples/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 712
  Batch size = 8


In [ ]:
gold = list(X_validation["label"])
evaluate(pred_list, gold)

RMSE = 66.267892
Pearson: 0.278467
